In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
import importlib
importlib.reload(tf)

<module 'tensorflow' from '/Users/jinli/anaconda3/lib/python3.6/site-packages/tensorflow/__init__.py'>

In [2]:
data_path = "../../data/cull%i/model_data/" % 1

In [3]:
devtest_aa_dict = np.load(data_path + 'devtest_aa_dict.npy')[()]
devtest_cmap_dict = np.load(data_path + 'devtest_cmap_dict.npy')[()]
train_aa_dict = np.load(data_path + 'train_aa_dict.npy')[()]
train_cmap_dict = np.load(data_path + 'train_cmap_dict.npy')[()]
valid_aa_dict = np.load(data_path + 'valid_aa_dict.npy')[()]
valid_cmap_dict = np.load(data_path + 'valid_cmap_dict.npy')[()]

In [110]:
# devtest_aa_dict

In [5]:
# from keras import backend as K
# from keras.engine.topology import Layer
# # from keras.layers import Layer

# # Need to create my own outer product layer
# class OuterConcat(Layer):
#     def __init__(self, output_dim, **kwargs):
#         self.output_dim = output_dim
#         self.name = "HII"
        
#     def built(self, input_shape):
#         # Create a trainable weight variable for this layer.
#         self.kernel = self.add_weight(name='kernel', 
#                                       shape=(input_shape[1], self.output_dim),
#                                       initializer='uniform',
#                                       trainable=True)
#         super(MyLayer, self).build(input_shape)  # Be sure to call this at the end
        
#     def call(self, x):
#         return K.dot(x, self.kernel)
    
#     def compute_output_shape(self, input_shape):
#         return (input_shape[0], self.output_dim)

In [6]:
class MyLayer(tf.keras.layers.Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return K.dot(x, self.kernel)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [7]:
# seq2pairwise(x)
# x = [1, 2, 3]
# y = [4, 5, 6]
# X, Y = tf.meshgrid(x, y)
# X.get_shape

In [8]:

def seq2pairwise(incoming):
    L = tf.shape(incoming)[1]
    #save the indexes of each position
    v = tf.range(0, L, 1)
    i, j = tf.meshgrid(v, v)
    m = (i+j)/2
    #switch batch dim with L dim to put L at first
    incoming2 = tf.transpose(incoming, perm=[1, 0, 2])
    #full matrix i with element in incomming2 indexed i[i][j]
    out1 = tf.nn.embedding_lookup(incoming2, i)
    out2 = tf.nn.embedding_lookup(incoming2, j)
    out3 = tf.nn.embedding_lookup(incoming2, m)
    #concatante final feature dim together
    out = tf.concat([out1, out2, out3], axis=3)
    #return to original dims
    output = tf.transpose(out, perm=[2, 0, 1, 3])
    return output

In [9]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs

    def build(self, input_shape):
        self.kernel = self.add_variable(
            "kernel",
            shape=[int(input_shape[-1]),
                   self.num_outputs])

    def call(self, input):
        return tf.matmul(input, self.kernel)


layer = MyDenseLayer(10)
print(layer)
print(layer(tf.zeros([10, 5])))
print(layer.trainable_variables)

Tensor("my_dense_layer/MatMul:0", shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32>]


In [10]:
class OuterProduct(tf.keras.layers.Layer):
    def __init__(self):
        super(OuterProduct, self).__init__()
    
#     def build(self, input_shape):
#         self.kernel = self.add_variable(
#             "kernel",
#             shape=[
#                 (input_shape[0]), 
#                 (input_shape[1]),
#                 (input_shape[1]),
#                 int(input_shape[2]) * 3
#                   ]
#         )
    def call(self, incoming):
        
        L = tf.shape(incoming)[1]
        
        #save the indexes of each position
        v = tf.range(0, L, 1)
        
        i, j = tf.meshgrid(v, v)
        
        m = tf.cast((i+j)/2, tf.int32)
        
        #switch batch dim with L dim to put L at first
        incoming2 = tf.transpose(incoming, perm=[1, 0, 2])
        
        #full matrix i with element in incomming2 indexed i[i][j]
        out1 = tf.nn.embedding_lookup(incoming2, i)
        out2 = tf.nn.embedding_lookup(incoming2, j)
        out3 = tf.nn.embedding_lookup(incoming2, m)
        
        #concatante final feature dim together
        out = tf.concat([out1, out2, out3], axis=3)
        #return to original dims
        output = tf.transpose(out, perm=[2, 0, 1, 3])
        return output

In [11]:
class ResidualBlock1D(tf.keras.layers.Layer):
    """
    All the layers in the residual block will have the 
    same number of features and stride.
    There are two layers in this block

    """

    def __init__(self, stride):
#         self.size = size
        self.stride = stride
        super(ResidualBlock1D, self).__init__()
        

    def call(self, x):
        size = int (x.shape[-1])
        y = tf.keras.layers.Conv1D(
            size,
            self.stride,
            activation='relu',
        )(x)
        y = tf.keras.layers.Conv1D(
            size,
            self.stride,
            activation='relu',
        )(y)
        
        y = tf.keras.layers.add([y, x])
        
        return y
    
class ResidualBlock2D(tf.keras.layers.Layer):
    """
    All the layers in the residual block will have the 
    same number of features and stride.
    There are two layers in this block

    """

    def __init__(self, stride):
#         self.size = size
        self.stride = stride
        super(ResidualBlock2D, self).__init__()
        

    def call(self, x):
        size = int (x.shape[-1])
        y = tf.keras.layers.Conv2D(
            size,
            self.stride,
            activation='relu',
            padding='same'
        )(x)
        y = tf.keras.layers.Conv2D(
            size,
            self.stride,
            activation='relu',
            padding='same'
        )(y)
        
        y = tf.keras.layers.add([y, x])
        
        return y

In [103]:
# from keras.models import Sequential

# model = Sequential()

# from keras import Input

# from keras.models import Model
# from tensorflow.keras.model import Model
# from keras import layers
# from tf.keras.layers

input_tensor = tf.keras.Input(shape=(None, 20))
x = tf.keras.layers.Conv1D(
    40,
    1,
    activation='relu',
)(input_tensor)

x = tf.keras.layers.Conv1D(
    60,
    1,
    activation='relu',
)(x)

x = ResidualBlock1D(1)(x)

"""
EDIT: did not have to do this lol.
Keras 2.2.4

To make this work, I had to go to:
/Users/jinli/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer.py

and change on line: 1749
layer.outbound_nodes.append(self)

to

layer._outbound_nodes.append(self)
"""

x = OuterProduct()(x)
x = ResidualBlock2D(1)(x)
# x = tf.keras.layers.Dense(128, activation='relu')(x)


x = tf.keras.layers.Conv2D(2, 1, activation='relu', padding='same')(x)
# x = tf.keras.layers.Flatten()(x)
model = tf.keras.models.Model(
    input_tensor,
    x
)

# x = MyDenseLayer(10)(x)

# third column is the number of feature maps
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, None, 20)          0         
_________________________________________________________________
conv1d_28 (Conv1D)           (None, None, 40)          840       
_________________________________________________________________
conv1d_29 (Conv1D)           (None, None, 60)          2460      
_________________________________________________________________
residual_block1d_7 (Residual (None, None, 60)          0         
_________________________________________________________________
outer_product_7 (OuterProduc (None, None, None, 180)   0         
_________________________________________________________________
residual_block2d_7 (Residual (None, None, None, 180)   0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, None, None, 2)     362       
Total para

In [107]:
model.compile(
    optimizer=tf.train.AdamOptimizer(0.001),
    loss="sparse_categorical_crossentropy",
    sample_weight_mode="temporal"
)
model.fit_generator(
    aa_generator(train_aa_dict, train_cmap_dict),
    steps_per_epoch=10, 
    epochs=1
)

/Users/jinli/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
 4/10 [===========>..................] - ETA: 4s - loss: 0.7478

InvalidArgumentError: logits and labels must have the same first dimension, got logits shape [24649,2] and labels shape [24336]
	 [[{{node loss_24/conv2d_23_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}} = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"](loss_24/conv2d_23_loss/Reshape_1, loss_24/conv2d_23_loss/Cast)]]

In [ ]:
test = OuterProduct()
test1 = test(tf.zeros([4, 5, 6]))

In [109]:
# int (x.shape[-1])

# output and input from preprocessing probably doesn't align
import math
math.sqrt(24649)
math.sqrt(24336)

156.0

In [ ]:
# test1.shape
tf.VERSION

In [ ]:
zeros = tf.zeros([10, 5, 6])
type(test1)

In [75]:
# model1 = tf.keras.models.Sequential([
#     tf.keras.layers.Flatten(input_shape=(28, 28)),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(10, activation='softmax'),
# #     MyDenseLayer(10),
# #     test,
# ])

# # model2 = tf.keras.models.Sequential([
# #     test()
# # ])

In [ ]:
# model2.summary()
model1.compile(
    optimizer=tf.train.AdamOptimizer(0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
1 + 3 / 2

In [73]:
# from keras import models
# from keras import layers
# network = tf.keras.models.Sequential()
# network.add(tf.keras.layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
# network.add(tf.keras.layers.Dense(10, activation='softmax'))

# network.compile(optimizer='rmsprop',
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'])

In [74]:
# import tensorflow as tf
# from tensorflow.keras import layers

# model = tf.keras.Sequential([
#     # Adds a densely-connected layer with 64 units to the model:
#     layers.Dense(64, activation='relu', input_shape=(32,)),
#     # Add another:
#     layers.Dense(64, activation='relu'),
#     # Add a softmax layer with 10 output units:
#     layers.Dense(10, activation='softmax')])

# model.compile(
#     optimizer=tf.train.AdamOptimizer(0.001),
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )



In [93]:
import random
def aa_generator(x, y):
    """
    
    """
    
    inputs = x.copy()
    outputs = y.copy()
    keys = set (x.keys())
#     print(keys)
    while True:
        try:
            key = random.sample(keys, 1)[0]
            keys.remove(key)
            
            one_hot_aa = x[key]
            one_hot_aa = np.reshape(one_hot_aa, (1,) + one_hot_aa.shape)
            cmap = y[key]
            cmap = np.reshape(cmap, (1,) + cmap.shape + (1,))
            yield one_hot_aa, cmap

#             yield key
        except ValueError:
            # if out of keys, reinsert back the keys
            inputs = x.copy()
            outputs = y.copy()
            keys = set (x.keys())

In [65]:
i = 0
keys = []
for item in aa_generator(train_aa_dict, train_cmap_dict):
    if i == 142:
        break
    keys.append(item)
    i += 1

In [66]:
# train_aa_dict['1']
# keys
# len (aa_generator)
len (set (keys))

140

In [83]:
# len (train_aa_dict.keys())
array = train_aa_dict['16vp']#.reshape(-1)
np.reshape(array, (1,) + array.shape).shape

(1, 366, 20)